#Wykonał
##Daniel Sobczak
###215342
> Poniższy notebook stanowi zapis z wykonanego przeze mnie zadania na konkurs na platformie kaggle.com. Wybrałem konkurs pod tytułem "Titanic", który polega na stworzeniu modelu mającego na celu przewidzieć, który z podróżnych legendarnego statku Titanic, przeżyje katastrofę, która spotkała ten okręt. 
> Na podstawie bazy danych train.csv, zawierającej informacje o podróżnych oraz o tym czy przeżyli katastrofę, należy oszacować kto z bazy danych test.csv przeżyje katastrofę.
> Poniższe komórki z kodem zawierają kolejno instrukcje odpowiadające za zaimportowanie baz danych, ich obsługę oraz stworzenie modelu. 
> Wynik programu (na platformie kaggle.com używa się określenia „submission”) w postaci pliku .cvm importuje się na platformę kaggle.com i otrzymuje procentowy wynik dopasowania utworzonej przez nas bazy do rzeczywistych wyników.



> Poniższa komórka kodu odpowiada za zaimportowanie wymaganych bibliotek "numpy" oraz "pandas".

In [65]:
import numpy as np # to linear algebra
import pandas as pd # to data processing

> Poniższa komórka kodu odpowiada za zaimportowanie bazy danych służącej do wytrenowania modelu. Dodatkowo wyświetlany jest początek bazy w celu zweryfikowania poprawności kodu i ścieżki. Plik jest importowany z mojego repozytorium, z folderu stworzonego w celu magazynowania plików związanych z tym zadaniem. 

In [71]:
train_data = pd.read_csv("https://raw.githubusercontent.com/daniel27a/missp/master/kaggle%20competition/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


> Tak jak w przypadku bazy danych zawierającej dane do wytrenowania modelu, tak samo w przypadku bazy danych służącej do testowania modelu, importujemy bazę oraz sprawdzamy poprawność kodu i ścieżki.

In [79]:
test_data = pd.read_csv("https://raw.githubusercontent.com/daniel27a/missp/master/kaggle%20competition/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


> Poniższe dwie komórki kodu odpowiadają za wyświetlenie najważniejszych danych statystycznych obu używanych baz danych.

In [73]:
print("Train data:")
train_data.describe()


Train data:


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [75]:
print("Test data:")
test_data.describe()

Test data:


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


> Poniższa komórka kodu ma za zadanie sprawdzić typy danych w poszczególnych kolumnach, oraz ilość niepustych wartości w obu bazach danych.

In [74]:
train_data.info()
print()
print(50*"=")
print()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass

> Wynik powyższej komórki kodu wskazuje na to, że nie wszystkie rekordy są wypełnione. Oznacza to, że puste rekordy należy uzupełnić, lub je usunąć. Poniższa komórka przedstawia w czytelny sposób ilość brakujących danych w kolumnach zarówno bazy trenującej jak i testowej.

In [76]:
print("Before data cleaning:")
print("Miss value in train_data:")
missing_val_count_by_column = (train_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])
print()
print("Miss value in test_data:")
missing_val_count_by_column = (test_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Before data cleaning:
Miss value in train_data:
Age         177
Cabin       687
Embarked      2
dtype: int64

Miss value in test_data:
Age       86
Fare       1
Cabin    327
dtype: int64


> Gdy znamy już rekordy, w których brakuje danych, możemy przejść do ich wypełniania lub usuwania. Zarówno w bazie trenującej jak i testowej, brakuje informacji na temat wieku podróżnych. W tym celu wypełniamy puste wartości medianą wszystkich podróżnych w danej bazie. W bazie testowej, brakuje jednej wartości opłaty za bilet, w tym celu również wypełniamy ją medianą. W bazie trenującej, brakuje dwóch wartości informującej o porcie, w którym pasażerowie wsiedli na statek. W tym przypadku usuwamy rekordy nie posiadających tych informacji. Dodatkowo zmieniono typ danych w kolumnie "Embarked" w bazie trenującej, tak aby mogła zostać obsłużona przez program.

In [77]:
train_data.Age = train_data.Age.fillna(train_data.Age.median())

train_data.dropna(subset=['Embarked'],inplace = True)
one_hot_encoded_training_predictors = pd.get_dummies(train_data.Embarked)

test_data.Age = test_data.Age.fillna(test_data.Age.median())
test_data.Fare = test_data.Fare.fillna(test_data.Fare.median())

> Po wyczyszczeniu i wypełnieniu baz danych, puste wartości zawierają jedynie kolumny informujące o zajmowanej przez podróżnych kabinie. Te kolumny nie zostały przeze mnie uzupełnione ponieważ nie wykorzystuje ich przy tworzeniu modelu.

In [78]:
print("After data cleaning:")
print("Miss value in train_data:")
missing_val_count_by_column = (train_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])
print()
print("Miss value in test_data:")
missing_val_count_by_column = (test_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])


After data cleaning:
Miss value in train_data:
Cabin    687
dtype: int64

Miss value in test_data:
Cabin    327
dtype: int64


> Po przygotowaniu baz danych, przechodzimy do tworzenia modelu. W moim przypadku, model zostaje nauczony metodą lasów losowych. Model, w celu przewidzenia prawdopodobieństwa przeżycia pasażerów, korzysta z takich danych jak: klasa biletu, płeć, wiek, ilość rodzeństwa na statku, ilość rodziców lub opiekunów na statku, wysokość opłaty za bilet oraz port w którym pasażer wsiadł na statek. Po porównaniu wyników z kilku modeli o różnych parametrach, wybrałem te które pozwoliły mi na najlepsze dopasowanie modelu:
- Ilość drzew = 130,
- Maksymalna ilość rozgałęzień w drzewie = 7.

> Powyższe parametry pozwoliły na osiągnięcie dopasowania na poziomie 0.79186

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=130, max_depth=7, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
